# Create Required Azure Services
Before starting this section, make sure you have set up the global variables from the previous section. Once your global variables have been set up, you no longer have to set them up again, as they are stored in a .env environment file. 

If you delete your .env file or your Azure login session expires, you may need to re-run the setup process as shown in the [previous section](setup_environment.ipynb). Keep this in mind when proceeding onto the other sections in this sample.

## Get Global Variables
First, we will read the previously stored global variables.

In [ ]:
from env_variables import *

## Create Resource Group
A resource group is a container that holds related resources for an Azure solution. The resource group can include all the resources for the solution, or only those resources that you want to manage as a group. 
We will now create a new Azure resource group using the resource group name and location set in the [previous section](setup_environment.ipynb). Check that this name is present in the .env file, as this name will be auto-filled into the placeholders starting with the character "$".

First, we will tag your resource group with your username and "evs" so that it will be easy to find.

In [ ]:
import os
from sys import platform

if platform == "win32": #Windows
    userName = (os.environ['USERNAME']).lower()
else: #Linux or MacOS
    userName = (os.environ['USER']).lower()

set_key(envPath, "USERNAME", userName)

resourceTags = "Owner={} Project=evs".format(userName)

In [ ]:
!az group create --name $resourceGroupName --location  $resourceLocation --tag $resourceTags

> <span style="color:red; font-weight: bold"> [!WARNING] </span>  
> Even if the cell above finishes executing, it may take several seconds to minutes to have the changes reflected on Azure Datacenter. Wait for 2-3 minutes before proceeding to the next cell to avoid running into errors.

## Create Azure IoT Hub
Next, we will create an Azure IoT Hub.

In [ ]:
!az iot hub create --name $iotHubServiceName --resource-group $resourceGroupName --location $resourceLocation --sku S1

Run the cells below to save these Azure IoT Hub connection string for future use by storing it in the .env file. This string is used to identify the hub.



In [ ]:
iotHubConnString = !az iot hub connection-string show --hub-name $iotHubServiceName --output tsv

In [ ]:
tempVar = set_key(envPath, "IOT_HUB_CONN_STRING", iotHubConnString[0].rstrip())

Note: If .env file has the error like this "IOT_HUB_CONN_STRING='ERROR: The command requires the extension azure-iot. Unable to prompt for extension install confirmation as no tty available. Run \'az config set extension.use_dynamic_install=yes_without_prompt\' to allow installing extensions without prompt.'" 
Rerun above two cells only
iotHubConnString = !az iot hub connection-string show --hub-name $iotHubServiceName --output tsv
tempVar = set_key(envPath, "IOT_HUB_CONN_STRING", iotHubConnString[0].rstrip())


## Create IoT Edge Device Identity
To run LVA on the Edge, we will need to create an IoT Edge device to run our modules with AI capability. In the code below, we will instruct Azure IoT Hub to create a resource framework for this cloud-based IoT Edge device.

In [ ]:
# Install iot hub CLI extensions
!az extension add --name azure-iot

If we want to connect/match physical devices to this cloud-based Edge device later, the connection will be set using the IoT Edge device's connection string. Everything in this cloud-based Edge device will be cloned into the physical device, including all the IoT Edge modules. Thus, we will also be printing out and saving the cloud-based IoT Edge device's connection string for future use.

In [ ]:
# Create edge device if one does not already exist
tempVar = !az iot hub device-identity create --hub-name $iotHubServiceName --device-id $iotDeviceId --edge-enabled
iotEdgeDeviceConnString = !az iot hub device-identity connection-string show --hub-name $iotHubServiceName --device-id $iotDeviceId --output tsv

Run the cell below to save the IoT Edge Device Connection String in the environment file for future use

In [ ]:
tempVar = set_key(envPath, "IOT_EDGE_DEVICE_CONN_STRING", iotEdgeDeviceConnString[0].rstrip())

## Create Azure Storage Services
To store media assets onto Azure, we must also create a storage resource.

In [ ]:
!az storage account create --name $storageServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Standard_LRS --tag $resourceTags

## Next Steps
If all the code cells above have successfully finished running, please validate .env file - It should have all the requried values. 
Return to the Readme (4.1) page to continue.   